In [1]:
# https://towardsdatascience.com/combining-numerical-and-text-features-in-deep-neural-networks-e91f0237eea4
"""combine meta data with text in keras"""

'combine meta data with text in keras'

In [2]:
import spacy
import gensim
nlp = spacy.load('en')
import pandas as pd
import numpy as np
import re
import demoji
from googletrans import Translator
translator = Translator()
import wordninja

In [3]:
file_path = "/home/ayush/Documents/personal/research/aaai/hostility_classification/data/constraint_Hindi_Train - Sheet1.csv"
data = pd.read_csv(file_path)
data.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"defamation,offensive"
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,non-hostile


In [4]:
# making labels
y = []
label_mapping = {'non-hostile':0, 'hate':1,'offensive':1,'fake':1,'defamation':1}
for labels in data['Labels Set']:
    temp = list(set([str(i)for i in [label_mapping[x] for x in labels.split(',')]]))
    temp = " ".join(temp)
    #print(temp)
    y.append(temp)
y = [int(x) for x in y]
y[:10]

[1, 0, 0, 1, 0, 1, 0, 0, 0, 0]

In [5]:
hindi_stop_words = ["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]
len(hindi_stop_words)

225

In [6]:
def feature_analysis(tweet):
    # url
    urls = re.findall(r"(?:https?\://)\S+", tweet)
    ## remove url from tweet
    tweet = re.sub(r"(?:https?\://)\S+", "", tweet)
    
    #cleaning 
    pat = re.compile(r"([',./(;_`:|\-)!])")
    tweet = pat.sub("", tweet)
    tweet = tweet.rstrip("\n")
    tweet = re.sub('\n', ' ', tweet)
    tweet = tweet.replace('\u200b', '')
    tweet = tweet.replace('\u200c', '')
    tweet = tweet.replace('\u200d', '')
    
    # hashtag
    hashtag_words = re.findall(r'#([^\s]+)', tweet)
    ## remove hashtag
    tweet = re.sub(r'#([^\s]+)', "", tweet)
    
    #mentions
    mention_words = re.findall(r'@([^\s]+)', tweet)
    ## remove mentions
    tweet = re.sub(r'@([^\s]+)', '', tweet)
    
    #print(translator.translate('Jankari', dest='hi').text)
    # english words
    english_words = [word for word in tweet.split() if re.search(r'[a-zA-Z]', word)]
    ## remove english words
    tweet = " ".join([word for word in tweet.split() if word not in english_words])
    #tweet = " ".join([word if word not in english_words else translator.translate(" ".join(wordninja.split(word)), dest='hi').text for word in tweet.split()])
    #f(x) if condition else g(x) for x in sequence
    emojis = demoji.findall_list(tweet, desc = True)
    tweet = demoji.replace(tweet,"")
    tweet = " ".join([word for word in tweet.split() if word not in hindi_stop_words])
    return tweet, urls, hashtag_words, mention_words, english_words, emojis

In [7]:
from tqdm import tqdm

In [8]:
all_tweets= []
for tweet in tqdm(data['Post']):
    all_tweets.append(feature_analysis(tweet))

100%|██████████| 5728/5728 [00:07<00:00, 785.16it/s]


In [9]:
embedding_loc = "/home/ayush/Documents/personal/research/aaai/hostility_classification/fastText/cc.hi.300.bin"

In [10]:
import fasttext
import fasttext.util
ft = fasttext.load_model(embedding_loc)

In [11]:
all_tweets[0]

('मेरे देश हिन्दु निराले है। पक्के राम भक्त बाबर साले जय श्री राम',
 [],
 [],
 [],
 [],
 ['folded hands', 'folded hands'])

In [12]:
X = []
for tweet in all_tweets:
    X.append(ft.get_sentence_vector(tweet[0]))

In [13]:
X_train = X[:5500]
y_train = y[:5500]

X_test = X[5500:]
y_test = y[5500:]

In [14]:
# from sklearn.decomposition import PCA
# pca=PCA(n_components=50)
# pca.fit(X_train)

# X_train_pca = pca.transform(X_train)
# X_test_pca = pca.transform(X_test)

In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [30]:
params={'C':[0.01,0.1,1,10],'gamma':[0.01,0.1,1,5,10]}
clf=SVC(kernel='rbf')
cv=GridSearchCV(estimator=clf,param_grid=params,cv=4,verbose=2,n_jobs = -1,scoring='f1')

In [31]:
cv.fit(X_train,y_train)

Fitting 4 folds for each of 20 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   56.1s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  2.5min finished


GridSearchCV(cv=4, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10],
                         'gamma': [0.01, 0.1, 1, 5, 10]},
             scoring='f1', verbose=2)

In [33]:
predictions=cv.predict(X_test)

In [34]:
from sklearn.metrics import accuracy_score

In [35]:
accuracy_score(y_test,predictions)

0.8859649122807017

In [36]:
cv.best_params_

{'C': 10, 'gamma': 5}

In [37]:
from sklearn import metrics
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.88      0.91      0.89       117
           1       0.90      0.86      0.88       111

    accuracy                           0.89       228
   macro avg       0.89      0.89      0.89       228
weighted avg       0.89      0.89      0.89       228



In [22]:
# testing data

In [38]:
file_path = "/home/ayush/Documents/personal/research/aaai/hostility_classification/data/Constraint_Hindi_Valid - Sheet1.csv"
data = pd.read_csv(file_path)

In [39]:
# making labels
y = []
label_mapping = {'non-hostile':0, 'hate':1,'offensive':1,'fake':1,'defamation':1}
for labels in data['Labels Set']:
    temp = list(set([str(i)for i in [label_mapping[x] for x in labels.split(',')]]))
    temp = " ".join(temp)
    #print(temp)
    y.append(temp)
y = [int(x) for x in y]
y[:10]

[0, 1, 0, 0, 0, 0, 0, 1, 0, 0]

In [40]:
all_tweets= []
for tweet in data['Post']:
    all_tweets.append(feature_analysis(tweet))

In [41]:
X = []
for tweet in all_tweets:
    X.append(ft.get_sentence_vector(tweet[0]))

In [42]:
# from sklearn.decomposition import PCA
# #pca=PCA(n_components=50)

# X=pca.transform(X)

In [43]:
predictions=cv.predict(X)

In [44]:
accuracy_score(y,predictions)

0.8914919852034525

In [45]:
from sklearn import metrics
print(metrics.classification_report(y, predictions))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90       435
           1       0.88      0.89      0.88       376

    accuracy                           0.89       811
   macro avg       0.89      0.89      0.89       811
weighted avg       0.89      0.89      0.89       811

